In [1]:
import numpy as np
import pandas as pd

# 1. Calculate the lift by quintile (1/5th) for each of the 2 partisanship models that you created in the week 2 exercises. [5 points] 

In [2]:
#process the score output from week 2 partisanship decision tree model
fx_df_all = pd.read_csv('FX_indicators.csv', index_col='VOTER_ID')
fx_df_all['D3'].replace({'N': 0, 'Y': 1}, inplace=True)
question1_DecisionTree_score_df = pd.read_csv('question1_DecisionTree_score.csv', index_col='VOTER_ID')
df1 = pd.DataFrame({'SET_NO':fx_df_all['SET_NO'], 'D3': fx_df_all['D3'],
                   'score': question1_DecisionTree_score_df['D3']})
df1['quintile'] = pd.qcut(df1['score'], 5, labels=np.arange(1, 6, 1))
D3_total_percent = df1.loc[df1['D3'] == 1.0].shape[0]/ df1.shape[0]
df_test = df1.loc[df1['SET_NO'] == 3]
print('Quintile', '% D3', 'Lift')
for i in [5,4,3,2,1]:
    D3_percent = df_test.loc[(df_test['D3'] == 1.0) & (df_test['quintile'] == i)].shape[0] / df_test.loc[df_test['quintile'] == i].shape[0]
    print(i, round(D3_percent*100, 1), round(D3_percent/D3_total_percent, 2))

Quintile % D3 Lift
5 68.8 1.66
4 49.7 1.2
3 38.4 0.92
2 31.8 0.77
1 19.3 0.47


In [3]:
#process the score output from week 2 partisanship logistic model
question2_Logistic_score_df = pd.read_csv('question2_Logistic_score.csv', index_col='VOTER_ID')
df2 = pd.DataFrame({'SET_NO':fx_df_all['SET_NO'], 'D3': fx_df_all['D3'],
                   'score': question2_Logistic_score_df['D3']})
df2['quintile'] = pd.qcut(df2['score'], 5, labels=np.arange(1, 6, 1))
D3_total_percent = df2.loc[df2['D3'] == 1.0].shape[0]/ df2.shape[0]
df_test = df2.loc[df2['SET_NO'] == 3]
print('Quintile', '% D3', 'Lift')
for i in [5,4,3,2,1]:
    D3_percent = df_test.loc[(df_test['D3'] == 1.0) & (df_test['quintile'] == i)].shape[0] / df_test.loc[df_test['quintile'] == i].shape[0]
    print(i, round(D3_percent*100, 1), round(D3_percent/D3_total_percent, 2))

Quintile % D3 Lift
5 65.8 1.59
4 48.0 1.16
3 38.2 0.92
2 32.7 0.79
1 22.7 0.55


# 2. Combine the two partisanship models made in week 2 to create an ensemble model predicting partisanship. [5 points] 3. Calculate the quintile lift for the combined partisanship model. [5 points] 


In [4]:
#the simple way to ensemble model is to average the score output from the two partisanship models in week 2 
df_combine = pd.DataFrame({'SET_NO': df1['SET_NO'], 'D3': df1['D3'], 'score': (df1.score + df2.score)/2})
df_combine['quintile'] = pd.qcut(df_combine['score'], 5, labels=np.arange(1, 6, 1))
D3_total_percent = df_combine.loc[df_combine['D3'] == 1.0].shape[0]/ df_combine.shape[0]
df_test = df_combine.loc[df_combine['SET_NO'] == 3]
print('Quintile', '% D3', 'Lift')
for i in [5,4,3,2,1]:
    D3_percent = df_test.loc[(df_test['D3'] == 1.0) & (df_test['quintile'] == i)].shape[0] / df_test.loc[df_test['quintile'] == i].shape[0]
    print(i, round(D3_percent*100, 1), round(D3_percent/D3_total_percent, 2))

Quintile % D3 Lift
5 69.4 1.67
4 49.5 1.19
3 38.6 0.93
2 30.5 0.74
1 19.5 0.47


# 4. Build one or more models predicting candidate support, rather than partisanship. 

In [5]:
#we use one of the CAND... DV 'CAND1_SD2' as the dependent variable to predict candidate support, we use decision tree here.
#since we use wave 1 id, we will suppress MSG_A and MSG_B.
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import tree
fx_df_all = pd.read_csv('FX_indicators.csv', index_col='VOTER_ID')
ind_list=['OPP_SEX','AGE','HH_ND','HH_NR','HH_NI','MED_AGE','MED_AGE_M','MED_AGE_F','NH_WHITE',
          'NH_AA','NH_NATAM','NH_ASIAN','NH_HPI','NH_OTHER','NH_MULT','HISP','COMM_LT10','COMM_609P',
          'MED_HH_INC','COMM_CAR','COMM_CP','COMM_PT','COMM_WALK','KIDS','KIDS_MC','M_NEV_MAR','M_MAR',
          'M_MAR_SP','M_MAR_SNP','F_NEV_MAR','F_MAR','F_MAR_SP','F_MAR_SNP','ED_ASSOC','ED_BACH','ED_MD',
          'ED_PROF','ED_DOC','ED_4COL','GENDER_F','GENDER_M','H_AFDLN3P','H_AFSSLN3P','H_F1','H_FFDLN2',
          'H_FFSLN2','H_M1','H_MFDLN2','H_MFDLN3P','H_MFSLN2','H_MFSLN3P','H_MFSSLN3P','H_MMDLN2','H_MMSLN2',
          'PARTY_D','PARTY_I','PARTY_R','HHP_D','HHP_DD','HHP_DI','HHP_DR','HHP_I','HHP_II','HHP_R','HHP_RI',
          'HHP_RR','VPP_12','VPP_16','VPR_12','VPR_14','VPR_16','VG_08','VG_10','VG_12','VG_14','VG_16','PP_PELIG',
          'PR_PELIG','AP_PELIG','G_PELIG','E_PELIG','NL5G','NL3PR','NL5AP','NL2PP','REG_DAYS','UPSCALEBUY',
          'UPSCALEMAL','UPSCALEFEM','BOOKBUYERI','FAMILYMAGA','FEMALEORIE','RELIGIOUSM','GARDENINGM','CULINARYIN',
          'HEALTHFITN','DOITYOURSE','FINANCIALM','RELIGIOUSC','POLITICALC','MEDIANEDUC','PREC_HRC','PREC_DT','MSG_A','MSG_B']
id_list=['CAND1S','CAND2S','MESSAGE']
dv_list=['VG14','D2','R2','D3','R3','I3','CAND1_UND','CAND1_SD2','CAND1_SDA','CAND1_LD2','CAND1_LDA',
         'CAND1_SR2','CAND1_SRA','CAND1_LRA','CAND1_LR2','CAND1_SFT','CAND2_UND','CAND2_SD2','CAND2_SDA',
         'CAND2_LD2','CAND2_LDA','CAND2_SR2','CAND2_SRA','CAND2_LRA','CAND2_LR2','CAND2_SFT','MOVED_RD',
         'MOVED_DR','MOVED_AW','MOVED_U','MOVED_AD','MOVED_AR','MOVED_RDMA','MOVED_DRMA','MOVED_AWMA',
         'MOVED_ADMA','MOVED_ARMA','MOVED_RDMB','MOVED_DRMB','MOVED_AWMB','MOVED_ADMB','MOVED_ARMB']
suppress_list =['MSG_A','MSG_B']
dv = 'CAND1_SD2'
model = 'DecisionTree'
question = 'question4_CAND1_SD2'
fx_df_all.drop(id_list, axis=1, inplace=True)
fx_df_all.drop(suppress_list, axis=1, inplace=True)
temp = dv_list[:]
temp.remove(dv)
fx_df_all.drop(temp, axis=1, inplace=True)
fx_df_all[dv].replace({'N': 0, 'Y': 1}, inplace=True)
fx_df = fx_df_all.dropna(subset=[dv])
df_train = fx_df.loc[(fx_df['SET_NO'] == 1) | (fx_df['SET_NO'] == 2)]
df_train.drop(['SET_NO'], axis=1, inplace=True) # Remove `SET_NO` as a indicator
y_train = df_train.pop(dv).values
X_train = df_train
df_test = fx_df.loc[fx_df['SET_NO'] == 3]
df_test.drop(['SET_NO'], axis=1, inplace=True) # Remove `SET_NO` as a indicator
y_test = df_test.pop(dv).values
X_test = df_test
clf = tree.DecisionTreeClassifier(min_samples_leaf=500)
clf.fit(X_train, y_train) # Fit the classifier using the training set
y_pred = clf.predict(X_test) # Predict the labels of the test set
report = classification_report(y_test, y_pred)
filename = '{}_{}_classification.txt'.format(question, model)
with open(filename, 'w') as f:
    f.write(report)
# Remove extraneous fields (the DV and `SET_NO`) before scoring.
fx_df_all.drop([dv, 'SET_NO'], axis=1, inplace=True)
# Score the whole file.
pred = clf.predict_proba(fx_df_all.values)[:, 1]
scores = pd.DataFrame({dv: pred*100}, index=fx_df_all.index)
# Write a CSV file of scores.
filename = '{}_{}_score.csv'.format(question, model)
scores.to_csv(filename)

C:\Jenny\DataScience\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# 5. Build a model predicting the overall persuadability of voters in FX [5 points] 

In [6]:
#we use one of the MOVED... DV 'MOVED_AW' as the dependent variable to predict overall persuadability of voters.

import pandas as pd
from sklearn.metrics import classification_report
from sklearn import tree
fx_df_all = pd.read_csv('FX_indicators.csv', index_col='VOTER_ID')
ind_list=['OPP_SEX','AGE','HH_ND','HH_NR','HH_NI','MED_AGE','MED_AGE_M','MED_AGE_F','NH_WHITE',
          'NH_AA','NH_NATAM','NH_ASIAN','NH_HPI','NH_OTHER','NH_MULT','HISP','COMM_LT10','COMM_609P',
          'MED_HH_INC','COMM_CAR','COMM_CP','COMM_PT','COMM_WALK','KIDS','KIDS_MC','M_NEV_MAR','M_MAR',
          'M_MAR_SP','M_MAR_SNP','F_NEV_MAR','F_MAR','F_MAR_SP','F_MAR_SNP','ED_ASSOC','ED_BACH','ED_MD',
          'ED_PROF','ED_DOC','ED_4COL','GENDER_F','GENDER_M','H_AFDLN3P','H_AFSSLN3P','H_F1','H_FFDLN2',
          'H_FFSLN2','H_M1','H_MFDLN2','H_MFDLN3P','H_MFSLN2','H_MFSLN3P','H_MFSSLN3P','H_MMDLN2','H_MMSLN2',
          'PARTY_D','PARTY_I','PARTY_R','HHP_D','HHP_DD','HHP_DI','HHP_DR','HHP_I','HHP_II','HHP_R','HHP_RI',
          'HHP_RR','VPP_12','VPP_16','VPR_12','VPR_14','VPR_16','VG_08','VG_10','VG_12','VG_14','VG_16','PP_PELIG',
          'PR_PELIG','AP_PELIG','G_PELIG','E_PELIG','NL5G','NL3PR','NL5AP','NL2PP','REG_DAYS','UPSCALEBUY',
          'UPSCALEMAL','UPSCALEFEM','BOOKBUYERI','FAMILYMAGA','FEMALEORIE','RELIGIOUSM','GARDENINGM','CULINARYIN',
          'HEALTHFITN','DOITYOURSE','FINANCIALM','RELIGIOUSC','POLITICALC','MEDIANEDUC','PREC_HRC','PREC_DT','MSG_A','MSG_B']
id_list=['CAND1S','CAND2S','MESSAGE']
dv_list=['VG14','D2','R2','D3','R3','I3','CAND1_UND','CAND1_SD2','CAND1_SDA','CAND1_LD2','CAND1_LDA',
         'CAND1_SR2','CAND1_SRA','CAND1_LRA','CAND1_LR2','CAND1_SFT','CAND2_UND','CAND2_SD2','CAND2_SDA',
         'CAND2_LD2','CAND2_LDA','CAND2_SR2','CAND2_SRA','CAND2_LRA','CAND2_LR2','CAND2_SFT','MOVED_RD',
         'MOVED_DR','MOVED_AW','MOVED_U','MOVED_AD','MOVED_AR','MOVED_RDMA','MOVED_DRMA','MOVED_AWMA',
         'MOVED_ADMA','MOVED_ARMA','MOVED_RDMB','MOVED_DRMB','MOVED_AWMB','MOVED_ADMB','MOVED_ARMB']
party_list =[]
dv = 'MOVED_AW'
model = 'DecisionTree'
question = 'question5_MOVED_AW'
fx_df_all.drop(id_list, axis=1, inplace=True)
fx_df_all.drop(party_list, axis=1, inplace=True)
temp = dv_list[:]
temp.remove(dv)
fx_df_all.drop(temp, axis=1, inplace=True)
fx_df_all[dv].replace({'N': 0, 'Y': 1}, inplace=True)
fx_df = fx_df_all.dropna(subset=[dv])
df_train = fx_df.loc[(fx_df['SET_NO'] == 1) | (fx_df['SET_NO'] == 2)]
df_train.drop(['SET_NO'], axis=1, inplace=True) # Remove `SET_NO` as a indicator
y_train = df_train.pop(dv).values
X_train = df_train
df_test = fx_df.loc[fx_df['SET_NO'] == 3]
df_test.drop(['SET_NO'], axis=1, inplace=True) # Remove `SET_NO` as a indicator
y_test = df_test.pop(dv).values
X_test = df_test
clf = tree.DecisionTreeClassifier(min_samples_leaf=500)
clf.fit(X_train, y_train) # Fit the classifier using the training set
y_pred = clf.predict(X_test) # Predict the labels of the test set
report = classification_report(y_test, y_pred)
filename = '{}_{}_classification.txt'.format(question, model)
with open(filename, 'w') as f:
    f.write(report)
# Remove extraneous fields (the DV and `SET_NO`) before scoring.
fx_df_all.drop([dv, 'SET_NO'], axis=1, inplace=True)
# Score the whole file.
pred = clf.predict_proba(fx_df_all.values)[:, 1]
scores = pd.DataFrame({dv: pred*100}, index=fx_df_all.index)
# Write a CSV file of scores.
filename = '{}_{}_score.csv'.format(question, model)
scores.to_csv(filename)

C:\Jenny\DataScience\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# 6. Build two uplift models predicting how likely it is that a voter will become more likely to support the Democratic candidate based on the test mailings for message A and message B. [5 points] 

In [7]:
#we use MOVED_RDMA and MOVED_RDMB as the dependent variable for the two models for message A and message B

import pandas as pd
from sklearn.metrics import classification_report
from sklearn import tree
fx_df_all = pd.read_csv('FX_indicators.csv', index_col='VOTER_ID')
ind_list=['OPP_SEX','AGE','HH_ND','HH_NR','HH_NI','MED_AGE','MED_AGE_M','MED_AGE_F','NH_WHITE',
          'NH_AA','NH_NATAM','NH_ASIAN','NH_HPI','NH_OTHER','NH_MULT','HISP','COMM_LT10','COMM_609P',
          'MED_HH_INC','COMM_CAR','COMM_CP','COMM_PT','COMM_WALK','KIDS','KIDS_MC','M_NEV_MAR','M_MAR',
          'M_MAR_SP','M_MAR_SNP','F_NEV_MAR','F_MAR','F_MAR_SP','F_MAR_SNP','ED_ASSOC','ED_BACH','ED_MD',
          'ED_PROF','ED_DOC','ED_4COL','GENDER_F','GENDER_M','H_AFDLN3P','H_AFSSLN3P','H_F1','H_FFDLN2',
          'H_FFSLN2','H_M1','H_MFDLN2','H_MFDLN3P','H_MFSLN2','H_MFSLN3P','H_MFSSLN3P','H_MMDLN2','H_MMSLN2',
          'PARTY_D','PARTY_I','PARTY_R','HHP_D','HHP_DD','HHP_DI','HHP_DR','HHP_I','HHP_II','HHP_R','HHP_RI',
          'HHP_RR','VPP_12','VPP_16','VPR_12','VPR_14','VPR_16','VG_08','VG_10','VG_12','VG_14','VG_16','PP_PELIG',
          'PR_PELIG','AP_PELIG','G_PELIG','E_PELIG','NL5G','NL3PR','NL5AP','NL2PP','REG_DAYS','UPSCALEBUY',
          'UPSCALEMAL','UPSCALEFEM','BOOKBUYERI','FAMILYMAGA','FEMALEORIE','RELIGIOUSM','GARDENINGM','CULINARYIN',
          'HEALTHFITN','DOITYOURSE','FINANCIALM','RELIGIOUSC','POLITICALC','MEDIANEDUC','PREC_HRC','PREC_DT','MSG_A','MSG_B']
id_list=['CAND1S','CAND2S','MESSAGE']
dv_list=['VG14','D2','R2','D3','R3','I3','CAND1_UND','CAND1_SD2','CAND1_SDA','CAND1_LD2','CAND1_LDA',
         'CAND1_SR2','CAND1_SRA','CAND1_LRA','CAND1_LR2','CAND1_SFT','CAND2_UND','CAND2_SD2','CAND2_SDA',
         'CAND2_LD2','CAND2_LDA','CAND2_SR2','CAND2_SRA','CAND2_LRA','CAND2_LR2','CAND2_SFT','MOVED_RD',
         'MOVED_DR','MOVED_AW','MOVED_U','MOVED_AD','MOVED_AR','MOVED_RDMA','MOVED_DRMA','MOVED_AWMA',
         'MOVED_ADMA','MOVED_ARMA','MOVED_RDMB','MOVED_DRMB','MOVED_AWMB','MOVED_ADMB','MOVED_ARMB']
party_list =[]
dv = 'MOVED_RDMB'
model = 'DecisionTree'
question = 'question6_MOVED_RDMB'
fx_df_all.drop(id_list, axis=1, inplace=True)
fx_df_all.drop(party_list, axis=1, inplace=True)
temp = dv_list[:]
temp.remove(dv)
fx_df_all.drop(temp, axis=1, inplace=True)
fx_df_all[dv].replace({'N': 0, 'Y': 1}, inplace=True)
fx_df = fx_df_all.dropna(subset=[dv])
df_train = fx_df.loc[(fx_df['SET_NO'] == 1) | (fx_df['SET_NO'] == 2)]
df_train.drop(['SET_NO'], axis=1, inplace=True) # Remove `SET_NO` as a indicator
y_train = df_train.pop(dv).values
X_train = df_train
df_test = fx_df.loc[fx_df['SET_NO'] == 3]
df_test.drop(['SET_NO'], axis=1, inplace=True) # Remove `SET_NO` as a indicator
y_test = df_test.pop(dv).values
X_test = df_test
clf = tree.DecisionTreeClassifier(min_samples_leaf=500)
clf.fit(X_train, y_train) # Fit the classifier using the training set
y_pred = clf.predict(X_test) # Predict the labels of the test set
report = classification_report(y_test, y_pred)
filename = '{}_{}_classification.txt'.format(question, model)
with open(filename, 'w') as f:
    f.write(report)
# Remove extraneous fields (the DV and `SET_NO`) before scoring.
fx_df_all.drop([dv, 'SET_NO'], axis=1, inplace=True)
# Score the whole file.
pred = clf.predict_proba(fx_df_all.values)[:, 1]
scores = pd.DataFrame({dv: pred*100}, index=fx_df_all.index)
# Write a CSV file of scores.
filename = '{}_{}_score.csv'.format(question, model)
scores.to_csv(filename)

C:\Jenny\DataScience\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Jenny\DataScience\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 7. [Applied Predictive Analytics Only] [20 points] a. Perform the lift calculation from question 1 for the models built last week using the small and full datasets. b. How does the lift differ in models built using the two different datasets? 


In [8]:
#we use the full and small dataset output score file here from last week
#we can see the lift is better for the full dataset score output.
fx_df_all = pd.read_csv('FX_indicators.csv', index_col='VOTER_ID')
fx_df_all['D3'].replace({'N': 0, 'Y': 1}, inplace=True)
question1_DecisionTree_score_df = pd.read_csv('question1_DecisionTree_score.csv', index_col='VOTER_ID')
df1 = pd.DataFrame({'SET_NO':fx_df_all['SET_NO'], 'D3': fx_df_all['D3'],
                   'score': question1_DecisionTree_score_df['D3']})
df1['quintile'] = pd.qcut(df1['score'], 5, labels=np.arange(1, 6, 1))
D3_total_percent = df1.loc[df1['D3'] == 1.0].shape[0]/ df1.shape[0]
df_test = df1.loc[df1['SET_NO'] == 3]
print('Quintile', '% D3', 'Lift')
for i in [5,4,3,2,1]:
    D3_percent = df_test.loc[(df_test['D3'] == 1.0) & (df_test['quintile'] == i)].shape[0] / df_test.loc[df_test['quintile'] == i].shape[0]
    print(i, round(D3_percent*100, 1), round(D3_percent/D3_total_percent, 2))

Quintile % D3 Lift
5 68.8 1.66
4 49.7 1.2
3 38.4 0.92
2 31.8 0.77
1 19.3 0.47


In [9]:
fx_df_all = pd.read_csv('FX_indicators.csv', index_col='VOTER_ID')
fx_df_all['D3'].replace({'N': 0, 'Y': 1}, inplace=True)
question1_DecisionTree_score_df = pd.read_csv('question1small_DecisionTree_score.csv', index_col='VOTER_ID')
df1 = pd.DataFrame({'SET_NO':fx_df_all['SET_NO'], 'D3': fx_df_all['D3'],
                   'score': question1_DecisionTree_score_df['D3']})
df1['quintile'] = pd.qcut(df1['score'], 5, labels=np.arange(1, 6, 1))
D3_total_percent = df1.loc[df1['D3'] == 1.0].shape[0]/ df1.shape[0]
df_test = df1.loc[df1['SET_NO'] == 3]
print('Quintile', '% D3', 'Lift')
for i in [5,4,3,2,1]:
    D3_percent = df_test.loc[(df_test['D3'] == 1.0) & (df_test['quintile'] == i)].shape[0] / df_test.loc[df_test['quintile'] == i].shape[0]
    print(i, round(D3_percent*100, 1), round(D3_percent/D3_total_percent, 2))

Quintile % D3 Lift
5 61.8 1.49
4 49.7 1.2
3 37.0 0.89
2 34.7 0.84
1 29.7 0.72
